In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD, Nadam, RMSprop
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.regularizers import l1, l2

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from models.vgg16 import vgg

Using Theano backend.
DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmp0z1as8/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmp0z1as8/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)
C:\Users\Tanuj\Anaconda2\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
lr=0.001
l1 = 0.00001
l2 = 0.00001
dropout = 0.5
n_epochs=500
batch_size=32
input_shape=(140, 140, 3)

name = 'vgg_140_rgb_corrected_lr_%f_sgd_he_normal__l1_%f_l2_%f_dropout_%f_r' % (lr, l1, l2, dropout)
# scheduled_5

In [3]:
def schedule(i):
    global lr
    if i > 0:
        return lr / 2 ** (i / 5)
    
    return lr

lr_scheduler = LearningRateScheduler(schedule)

In [4]:
print('loading model...')
# model = cnn(input_shape=input_shape, init='he_normal')
model = vgg(input_shape=input_shape)
model.summary()

optimizer = SGD(lr=lr, clipnorm=4., nesterov=True)
# optimizer = Nadam(lr=lr)
# optimizer = RMSprop(lr=lr)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

csv_logger = CSVLogger('%s_training.log' % name)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_current.hdf5" % name), verbose=0)

loading model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 140, 140, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 140, 140, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 140, 140, 64)  0           block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 140, 140, 64)  36928       activation_1[0][0]               
__________________________________________________________________________

In [5]:
print('Initializing data generators...')

train_set_file = config.corrected_train_data_file
validation_set_file = config.corrected_validation_data_file
test_set_file = config.corrected_test_data_file
# train_set_file = os.path.join(config.dataset_dir, 'corrected', '19', 'train_data.csv')
# validation_set_file = os.path.join(config.dataset_dir, 'corrected', '19', 'validation_data.csv')
# test_set_file = os.path.join(config.dataset_dir, 'corrected', '19', 'test_data.csv')

train_data_gen = DataGenerator(dataset_file=train_set_file, batch_size=batch_size)
validation_data_gen = DataGenerator(dataset_file=validation_set_file, batch_size=batch_size)
test_data_gen = DataGenerator(dataset_file=test_set_file, batch_size=batch_size)
print('done.')

Initializing data generators...
done.


In [6]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, best_model_checkpointer, current_model_checkpointer],
                              nb_worker=2)
print('done.')
# , lr_scheduler

Fitting model...
Epoch 1/500
10496/10496 [==============================] - 507s - loss: 0.6765 - acc: 0.6641 - val_loss: 0.6615 - val_acc: 0.6734
Epoch 2/500
10496/10496 [==============================] - 508s - loss: 0.6555 - acc: 0.6647 - val_loss: 0.6468 - val_acc: 0.6717
Epoch 3/500
10496/10496 [==============================] - 508s - loss: 0.6458 - acc: 0.6647 - val_loss: 0.6391 - val_acc: 0.6730
Epoch 4/500
10496/10496 [==============================] - 508s - loss: 0.6414 - acc: 0.6647 - val_loss: 0.6356 - val_acc: 0.6730
Epoch 5/500
10496/10496 [==============================] - 508s - loss: 0.6394 - acc: 0.6647 - val_loss: 0.6329 - val_acc: 0.6747
Epoch 6/500
10496/10496 [==============================] - 507s - loss: 0.6385 - acc: 0.6647 - val_loss: 0.6323 - val_acc: 0.6743
Epoch 7/500
10496/10496 [==============================] - 508s - loss: 0.6382 - acc: 0.6647 - val_loss: 0.6319 - val_acc: 0.6743
Epoch 8/500
10496/10496 [==============================] - 504s - loss: 0

KeyboardInterrupt: 

In [7]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])

Evaluating model...
done.
Test score: 0.634301039535
Test accuracy: 0.669894366197
